# Topic Modelling for News

![](https://images.unsplash.com/photo-1495020689067-958852a7765e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80)

Photo by [Roman Kraft](https://unsplash.com/photos/_Zua2hyvTBk)

This exercise is about modelling the main topics of a database of News headlines.

Begin by importing the needed libraries:

In [9]:
# TODO: import needed libraries
import pandas as pd
import numpy as np
import nltk

Load the data in the file `random_headlines.csv`

In [4]:
# TODO: load the dataset
df = pd.read_csv("random_headlines.csv")
print(df.shape)
df.head()

(20000, 2)


,publish_date,headline_text
0,20120305,ute driver hurt in intersection crash
1,20081128,6yo dies in cycling accident
2,20090325,bumper olive harvest expected
3,20100201,replica replaces northernmost sign
4,20080225,woods targets perfect season


This is always a good idea to perform some EDA (exploratory data analytics) on a dataset...

In [6]:
# TODO: Perform a short EDA
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   publish_date   20000 non-null  int64 
 1   headline_text  20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


Now perform all the needed preprocessing on those headlines: case lowering, tokenization, punctuation removal, stopwords removal, stemming/lemmatization.

In [14]:
# TODO: Preprocess the input data

# tokenize
from nltk.tokenize import word_tokenize
df["tokenize"] = df["headline_text"].apply(lambda row: word_tokenize(row))

# remove punctuation
df["alphanumeric"] = df["tokenize"].apply(lambda row: [word for word in row if word.isalpha()])

# remove stopwords
from nltk.corpus import stopwords
stop = stopwords.words("english")
df["stop"] = df["alphanumeric"].apply(lambda row: [word for word in row if word not in stop])

# stemming
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
df["stemmed"] = df["stop"].apply(lambda row: [stemmer.stem(word) for word in row])

df["stemmed"].head()

0    [ute, driver, hurt, intersect, crash]
1                       [die, cycl, accid]
2          [bumper, oliv, harvest, expect]
3    [replica, replac, northernmost, sign]
4          [wood, target, perfect, season]
Name: stemmed, dtype: object

Now use Gensim to compute a BOW

In [19]:
# TODO: Compute the BOW using Gensim
from gensim.corpora import Dictionary

dictionary = Dictionary(df["stemmed"])

BOW = [dictionary.doc2bow(document) for document in df["stemmed"]]

print(np.shape(BOW))
BOW[:2]

(20000,)


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(5, 1), (6, 1), (7, 1)]]

Compute the TF-IDF using Gensim

In [21]:
# TODO: Compute TF-IDF
from gensim.models import TfidfModel

tfidf_model = TfidfModel(BOW)
tfidf = tfidf_model[BOW]

print(np.shape(tfidf))
tfidf

(20000,)


Finally compute the **LSA** (also called LSI) using Gensim, for a given number of Topics that you choose yourself

In [23]:
# TODO: Compute LSA
from gensim.models import LsiModel

lsi_model = LsiModel(BOW, id2word=dictionary, num_topics=4)

For each of the topic, show the most significant words.

In [24]:
# TODO: Print the 3 or 4 most significant words of each topic
lsi_model.print_topics(num_topics=4, num_words=3)

[(0, '0.751*"polic" + 0.405*"man" + 0.208*"charg"'),
 (1, '0.669*"man" + -0.576*"polic" + 0.328*"charg"'),
 (2, '-0.654*"new" + -0.297*"plan" + 0.242*"man"'),
 (3, '-0.703*"new" + 0.345*"say" + 0.333*"plan"')]

What do you think about those results?

Now let's try to use LDA instead of LSA using Gensim

In [27]:
# TODO: Compute LDA
from gensim.models import LdaModel

lda_model = LdaModel(BOW, id2word=dictionary, num_topics=4)

In [28]:
# TODO: print the most frequent words of each topic
lda_model.print_topics(num_topics=4, num_words=3)

[(0, '0.009*"new" + 0.008*"murder" + 0.005*"plan"'),
 (1, '0.015*"polic" + 0.008*"new" + 0.007*"wa"'),
 (2, '0.008*"win" + 0.008*"interview" + 0.006*"day"'),
 (3, '0.011*"man" + 0.009*"court" + 0.009*"polic"')]

Now, how does it work with LDA?

The LDA model provides different answers due to attempting to optimise global minima but getting stuck in a local minimum.

Let's make some visualization of the LDA results using pyLDAvis.

In [35]:
# TODO: show visualization results of the LDA
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda_model, BOW, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.226164 -0.066639       1        1  25.621933
3     -0.096940 -0.035127       2        1  25.310149
1     -0.118821 -0.117368       3        1  24.728375
0     -0.010403  0.219133       4        1  24.339543, topic_info=        Term        Freq       Total Category  logprob  loglift
185    polic  637.000000  637.000000  Default  30.0000  30.0000
368       wa  190.000000  190.000000  Default  29.0000  29.0000
205   murder  201.000000  201.000000  Default  28.0000  28.0000
312    court  291.000000  291.000000  Default  27.0000  27.0000
139      man  470.000000  470.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
610     govt   88.153977  311.166539   Topic4  -5.6464   0.1518
857   welcom   70.863266  104.678847   Topic4  -5.8647   1.0229
428       sa   73.914466  153.956885   Topic4  -5.8225   0.6793
139      man   78.541686  470.464207   Topic4  -5.7618  -0.3770
439  council   75.786438  329.245052   Topic4  -5.7975  -0.0558

[308 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1103      3  0.993630   abbott
5         2  0.947839    accid
5         3  0.015288    accid
5         4  0.030575    accid
101       1  0.012960    accus
...     ...       ...      ...
151       1  0.815175    world
151       2  0.105973    world
151       3  0.057062    world
151       4  0.024455    world
3465      3  0.984409  zealand

[605 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1])

Depending on your results, you can try to fine tune the algorithm: number of topics, hyperparameters...
And check with others their results.